# Exercise #3 - Backpropagation and training

Finally, we are going to train our network so it can actually predict something useful.

You will first implement the whole backpropagation chain so we have the gradients of the loss with respect to the parameters of the neural network. Then we will implement the training loop and let it run for a while.

Let's start with the basic necessities.

In [ ]:
import numpy as np
from siouxdnn import dense, relu, sigmoid, binary_cross_entropy_loss

## Loss function

Since we are going from the back to the front of the network we start with the last block, the loss function. The loss function computes the loss based on the predictions of the network. So when we go back through the network we have to compute the partial derivative of the loss value with respect to those predictions. This is defined as follows.

$$\frac{\partial{L}}{\partial{\hat{y}}} = \frac{1}{M} \frac{\hat{y}-y}{\hat{y}(1-\hat{y})}$$

It's actually already a "chained" function. The $\frac{1}{M}$ part is to compensate for the `mean` used in the loss function, i.e. every sample only contributes for $\frac{1}{M}$th to the loss. The second part is the actual partial derivative of the loss function.

Implement this in the following function.

By the way, the clipping of `y_pred` is done to prevent division by zero.

In [ ]:
def binary_cross_entropy_loss_backward(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-7, 1-1e-7)
    m = y_pred.shape[0]
    #### BEGIN IMPLEMENTATION ####
    dy_pred = 
    #### END IMPLEMENTATION ####
    return dy_pred

Let's test your implementation.

In [ ]:
y_true = np.array([[1.0], [1.0], [0.0], [1.0], [0.0], [0.0], [0.0], [1.0], [1.0], [0.0]])
y_pred = np.array([[0.6], [0.4], [0.2], [0.7], [0.1], [0.2], [0.5], [0.9], [0.8], [0.6]])
dy_pred = binary_cross_entropy_loss_backward(y_true, y_pred)
print(dy_pred)

If you implemented it correctly the output should be:

    [[-0.16666667]
     [-0.25      ]
     [ 0.125     ]
     [-0.14285714]
     [ 0.11111111]
     [ 0.125     ]
     [ 0.2       ]
     [-0.11111111]
     [-0.125     ]
     [ 0.25      ]]

Otherwise, check your implementation.

With this single function we can already get some intuition on how training will work. The function `binary_cross_entropy_loss_backward` will return the gradient of the predictions. So if we change the predictions themselves slightly in the opposite direction, then the predictions will be closed to the ground truth.

Let's test this. We compute (and print) the loss at the start, then we do a number of small steps in the opposite direction of the gradient, followed by a final loss computation. The result should be that the loss ends up close to 0.

In [ ]:
from siouxdnn import binary_cross_entropy_loss

print(f'loss at start {binary_cross_entropy_loss(y_true, y_pred)}')
dy_pred = binary_cross_entropy_loss_backward(y_true, y_pred)

for _ in range(500):
    y_pred = y_pred - 1e-2*dy_pred
    dy_pred = binary_cross_entropy_loss_backward(y_true, y_pred)

print(f'loss at end {binary_cross_entropy_loss(y_true, y_pred)}')

The final training of the network will work in a similar fashion. Instead of changing the predictions we will then change the parameters `w` and `b` of each layer. But we need a few other building blocks before we can do that.

## Activation functions

Next up are the activation functions. These were defined as follows.

$$a_n = g(z_n)$$

During backpropagation we want to compute the following chain.

$$\frac{\partial{L}}{\partial{z_n}} = \frac{\partial{L}}{\partial{a_n}} \frac{\partial{a_n}}{\partial{z_n}} = \frac{\partial{L}}{\partial{a_n}} \frac{\partial{g(z_n)}}{\partial{z_n}}$$

### Sigmoid

Let's begin with backpropagating the `sigmoid` function. The sigmoid function has a nice feature that it's derivative can be described in terms of its own.

$$\frac{\partial{g(𝑧_n)}}{\partial{z_n}} = \frac{\partial{𝑠𝑖𝑔(𝑧_n)}}{\partial{z_n}} = 𝑠𝑖𝑔(𝑧_n)(1−𝑠𝑖𝑔(𝑧_n)) = a_n (1-a_n)$$

Implement it in the function below. Let it return the output of the whole chain, i.e. $\frac{\partial{L}}{\partial{z_n}}$!

In [ ]:
def sigmoid_backward(z, da):
    #### BEGIN IMPLEMENTATION ####
    a = 
    dz = 
    #### END IMPLEMENTATION ####
    return dz

Let's test your implementation.

In [ ]:
z = np.array([[ 0.90145683, 0.35448664, 0.12282405, 0.14473946],
              [ 0.90185776,-0.83611207,-0.74306445, 0.75256026],
              [-0.68165728, 0.46613136, 0.22696806,-0.34379635]])
da = np.array([[ 0.09298582,-0.13436976, 0.02718292, 0.02473067],
               [-0.12145152, 0.17550431,-0.03550441,-0.03230145],
               [-0.15272906, 0.22070212,-0.0446479 ,-0.04062008]])
dz = sigmoid_backward(z, da)
print(dz)

The output should be equal to:

    [[ 0.01909686 -0.03255884  0.00677016  0.0061504 ]
     [-0.02493875  0.03702021 -0.0077554  -0.00703186]
     [-0.03406903  0.0522837  -0.01101945 -0.00986076]]


If not, check your implementation.

### ReLU

The other activation function, ReLU, is even simpler. The ReLU function returns the input value `z` if it is greater than 0 otherwise it returns 0. That means that the gradient is 1 if `z` is greater than 0, otherwise it is 0.

$$\frac{\partial{g(𝑧_n)}}{\partial{z_n}} = \frac{\partial{relu(𝑧_n)}}{\partial{z_n}} = \begin{cases} 1, & z_n \gt 0 \\ 0, & z_n \le 0 \end{cases}$$

There are a few ways to compute the result. You could convert the z array to 1s and 0s using boolean logic, or you could set the `da` array values to 0 where applicable. Make sure you return a new array and not an in-place changed version of `da`, otherwise it might screw up other computations later on (see [np.copy](https://docs.scipy.org/doc/numpy-1.16.1/reference/generated/numpy.copy.html)).

Remember that this function also has to return the complete chain result $\frac{\partial{L}}{\partial{z_n}}$!

In [ ]:
def relu_backward(z, da):
    #### BEGIN IMPLEMENTATION ####
    dz = 
    #### END IMPLEMENTATION ####
    return dz

Testing, testing.

In [ ]:
z = np.array([[-0.3841685 , 0.96313277, 0.01752332, 0.70375869],
              [-0.90051653, 0.36881432,-0.87937234,-0.07766394],
              [ 0.99909255,-0.28151873, 0.21100903, 0.96628448]])
da = np.array([[ 0.10135023,-0.09567351, 0.00798541, 0.01288583],
               [ 0.12338271,-0.11647193, 0.00972135, 0.01568707],
               [ 0.18808731,-0.17755236, 0.01481944, 0.02391371]])
dz = relu_backward(z, da)
print(dz)

The output should be:

    [[ 0.         -0.09567351  0.00798541  0.01288583]
     [ 0.         -0.11647193  0.          0.        ]
     [ 0.18808731  0.          0.01481944  0.02391371]]

## Dense

And last but not least, the backpropagation of the `dense` function. This function is more involved, because it uses the parameters `w` and `b` and we are actually interested in the partial derivatives of those.

So instead of computing only one derivate this function should return 3.

One for the output of the previous layer, so we can continue the chain backwards.

$$\frac{\partial{L}}{\partial{a_{n-1}}} = \frac{\partial{L}}{\partial{z_n}} \frac{\partial{z_n}}{\partial{a_{n-1}}} = \frac{\partial{L}}{\partial{z_n}} w_{n}$$

And two for the parameters.

$$\frac{\partial{L}}{\partial{w_n}} = \frac{\partial{L}}{\partial{z_n}} \frac{\partial{z_n}}{\partial{w_n}} = \frac{\partial{L}}{\partial{z_n}} a_{n-1}$$

$$\frac{\partial{L}}{\partial{b_n}} = \frac{\partial{L}}{\partial{z_n}} \frac{\partial{z_n}}{\partial{b_n}} = \frac{\partial{L}}{\partial{z_n}} 1 = \frac{\partial{L}}{\partial{z_n}}$$

Thanks to backpropagation the term $\frac{\partial{L}}{\partial{z_n}}$ is already computed and given as input `dz`. Implement the rest in the function below.

Remember that the shapes of the outputs must match the shapes of the inputs. So `dw` must have the same shape as `w`, `db` the same as `b`, and `da_prev` the same as `a_prev`. Take real good care of this when you multiply matrices.

For instance, `dz` has shape `(#samples, #outputs)` and `w` has shape `(#inputs, #outputs)`. The output, `da_prev` must have shape `(#samples, #inputs)`. You cannot simply multiply `dz` with `w`, because then you would get `(#samples, #outputs)(#inputs, #outputs)` which is not valid. To correctly do this you have to transpose the matrix `w` (e.g. use `w.T`). Then you would get `(#samples, #outputs)(#outputs, #inputs)` which is valid and will return a matrix of shape `(#samples, #inputs)`.

Finally you have to do something smart with the gradient of `b`. Remember that `b` is added to each sample via the automatic 'broadcast'. For instance, if you have 20 samples and your layer has 6 outputs, then the result of $a_{n-1} w_n$ is a tensor with shape `(20, 6)`. The `b` parameter has shape `(1, 6)` and is broadcasted (i.e. repeated) to the shape `(20, 6)` before it is added. Since $\frac{\partial{L}}{\partial{b_n}}$ is equal to $\frac{\partial{L}}{\partial{z_n}}$ you have to "un-broadcast" it to get the right shape. In this case you have to use [np.sum](https://docs.scipy.org/doc/numpy-1.16.1/reference/generated/numpy.sum.html) with the right `axis` parameter and `keepdims=True`.

In [ ]:
def dense_backward(a_prev, dz, w):
    #### BEGIN IMPLEMENTATION ####
    dw = 
    db = 
    da_prev = 
    #### END IMPLEMENTATION ####
    return dw, db, da_prev

Let's test it. This one is big, because the dense layer uses the paramters `w` and `b`.

In [ ]:
a_prev = np.array([[ 0.35375751,-0.3776729 , 0.32942281,-0.27827347],
 [-0.93693319,-0.085921  , 0.01547009, 0.80320843],
 [-0.74478941, 0.22313203,-0.21071294, 0.86850543]])
dz = np.array([[ 0.        , 0.03582932, 0.        , 0.05249304,-0.08647048],
 [ 0.04884253, 0.06242368, 0.11415199, 0.09145607, 0.        ],
 [ 0.04169974, 0.05329477, 0.09745826, 0.07808142, 0.        ]])
w = np.array([[-0.89889975,-0.47854249,-0.52593611,-0.90020105, 0.63363267],
 [ 0.14985559,-0.36020986,-0.85147906,-0.68566762, 0.1925108 ],
 [-0.54756299, 0.80574929,-0.81764706,-0.4837164 , 0.66329728],
 [-0.6358417 ,-0.13549578,-0.08654791, 0.03401955,-0.01993571]])
dw, db, da_prev = dense_backward(a_prev, dz, w)
print(dw)
print(db)
print(da_prev)

The output should be:

    [[-0.07681971 -0.08550531 -0.17953867 -0.12527264 -0.03058958]
     [ 0.00510795 -0.0070035   0.01193801 -0.01026073  0.03265756]
     [-0.00803108  0.0015388  -0.01876978  0.00225447 -0.02848535]
     [ 0.07544718  0.08645567  0.17633087  0.12666501  0.02406244]]
    [[ 0.09054227  0.15154777  0.21161025  0.22203054 -0.08647048]]
    [[-0.11919067 -0.06554536 -0.05387793 -0.00134508]
     [-0.21614243 -0.1750728  -0.11402137 -0.04628258]
     [-0.18453349 -0.14946993 -0.09734674 -0.03951416]]

# Complete backpropagation

Now that we have all the building blocks available we can build the whole backpropagation chain for our model.

Below is the code of the model as it was implemented in the previous exercises. You need to implement the new `get_gradients` function.

In this function, first the forward pass is computed, so all the intermediate results are available. Then the backpropagation starts. Use the right builing blocks you implemented above with the right parameters.

You're almost done. Good luck!

In [ ]:
class Model(object):
    def __init__(self):
        N0, N1, N2,N3 = 5, 64, 64, 1
        self.w1 = np.random.uniform(-0.3, 0.3, size=(N0, N1))
        self.b1 = np.zeros((1, N1))
        self.w2 = np.random.uniform(-0.2, 0.2, size=(N1, N2))
        self.b2 = np.zeros((1, N2))
        self.w3 = np.random.uniform(-0.3, 0.3, size=(N2, N3))
        self.b3 = np.zeros((1, N3))
        
    def predict(self, x):
        a0 = x
        z1 = dense(a0, self.w1, self.b1)
        a1 = relu(z1)
        z2 = dense(a1, self.w2, self.b2)
        a2 = relu(z2)
        z3 = dense(a2, self.w3, self.b3)
        a3 = sigmoid(z3)
        y_pred = a3
        return y_pred

    def compute_loss(self, x, y_true):
        y_pred = self.predict(x)
        return binary_cross_entropy_loss(y_true, y_pred)

    def get_gradients(self, x, y_true):
        a0 = x

        z1 = dense(a0, self.w1, self.b1)
        a1 = relu(z1)
        z2 = dense(a1, self.w2, self.b2)
        a2 = relu(z2)
        z3 = dense(a2, self.w3, self.b3)
        a3 = sigmoid(z3)

        y_pred = a3

        loss = binary_cross_entropy_loss(y_true, y_pred)

        #### BEGIN IMPLEMENTATION ####
        dy_pred = 

        da3 = dy_pred

        dz3 = 
        dw3, db3, da2 = 
        dz2 = 
        dw2, db2, da1 = 
        dz1 = 
        dw1, db1, da0 = 

        dx = da0
        #### END IMPLEMENTATION ####

        return loss, dx, dw1, db1, dw2, db2, dw3, db3

We could add a test here if you implemented it right, but we need quite some data for that and we are almost done, so let's postpone that.

# Training

The model is implemented, we can predict values and compute the gradients. So we are finally ready to actually train the model. 

We will train the model on the same dataset as before. This time we will use both the training and the validation set. So let's import that first.

In [ ]:
from siouxdnn import load_data
X_train, Y_train, X_val, Y_val = load_data()
print('training set', X_train.shape, Y_train.shape)
print('validation set', X_val.shape, Y_val.shape)

Now we implement the training loop. This is actually pretty simple. You first have to compute the gradients using the function you just implemented. Then update all the parameters in the opposite direction with the `learning_rate` as a factor.

In [ ]:
def train(model, x, y_true, learning_rate):
    #### BEGIN IMPLEMENTATION ####
    loss, dx, dw1, db1, dw2, db2, dw3, db3 = 
    model.w1 = 
    model.b1 = 
    model.w2 = 
    model.b2 = 
    model.w3 = 
    model.b3 = 
    #### END IMPLEMENTATION ####
    return loss

Alright, this is it. Time to train your model. Run the following code and it will create a new model and run 1000 training steps, each time training on the whole training set at once (i.e. no batches).

A nice plot will be displayed while training. You should see both the training loss and the validation loss going down.

In [ ]:
%matplotlib notebook
from loss_plot import init_loss_plot, add_loss_to_plot, finish_loss_plot

from siouxdnn import reset_seed
reset_seed(123)
model = Model()

learning_rate = 1e-2

init_loss_plot()
for epoch in range(1000):
    train_loss = train(model, X_train, Y_train, learning_rate)
    val_loss = model.compute_loss(X_val, Y_val)

    add_loss_to_plot(train_loss, val_loss, epoch%50 == 0)
finish_loss_plot()

# Validation

The model is now trained!

Let's check how well it performs by inspecting the metrics on the validation dataset.

In [ ]:
from siouxdnn import get_metrics

y_pred = model.predict(X_val)

accuracy, precision, recall = get_metrics(Y_val, y_pred)
print(f'accuracy {accuracy:.3f}, precision {precision:.3f}, recall {recall:.3f}')

The output should be `accuracy 0.906, precision 0.884, recall 0.974`.

As you can see we do pretty well. An accuracy of 90.6% and high precision and recall.

# End

We are done. We have written a complete neural network (of 3 layers) all from scratch and trained it on a dataset. And it even performs quite well. Congratulations!

# 🥳